# Analyse BlueSeis BSPF Event - Demo 0

Here we analyse the data for the 6-DoF station BSPF that we fetch online with FDSN services.

In [1]:
import os
import matplotlib.pyplot as plt
from obspy import UTCDateTime

from sixdegrees.sixdegrees import sixdegrees
from sixdegrees.plots.plot_cwt_all import plot_cwt_all
from sixdegrees.plots.plot_waveform_cc import plot_waveform_cc
from sixdegrees.plots.plot_velocities import plot_velocities

## Configurations

Set configuration parameters for the event and data

In [2]:
config = {
    # Required time window parameters
    'tbeg': "2025-03-20 08:39:10",  # Start time
    'tend': "2025-03-20 08:39:40",  # End time
    
    # event
    'origin_time': "2025-03-20 03:39:05",
    'magnitude': 3.9,
    
    # Station coordinates (required)
    'station_lon': -116.455439,  # BSPF longitude
    'station_lat': 33.610643,    # BSPF latitude
    
    # SEED identifiers
    'seed': "XX.BSPF..",     # Default seed ID
    'rot_seed': ["PY.BSPF..HJZ", "PY.BSPF..HJN", "PY.BSPF..HJE"],  # Rotation sensor seed ID
    'tra_seed': ["PY.PFOIX..HHZ", "PY.PFOIX..HHN", "PY.PFOIX..HHE"],  # Translation sensor seed ID
    
    # data source
    'data_source': "fdsn",
    'fdsn_client_rot': "IRIS",
    'fdsn_client_tra': "IRIS",

    # Path configurations
    'workdir': "./",            # Working directory
    'path_to_data_out': "./data/",  # Output directory

    # Project information
    'project': "BSPF_Analysis",  # Project name
    
    # Remove response
    'tra_remove_response': True,
    'rot_remove_response': True,

    # Rotate to ZNE
    'rotate_zne': True,

    # Optional parameters
    'verbose': True,            # Print detailed information
}

Initalize a sixdegrees object

In [3]:
sd = sixdegrees(conf=config)

Store the configuration as a pickle file

In [4]:
filename = "./config/"+f"config_BSPF_M{str(config['magnitude'])}_{config['tbeg'][:10].replace('-','')}" 

sd.store_as_yaml(config, filename)

 -> file ./config/config_BSPF_M3.9_20250320.yml already exists. Aborting...


Get key information about the event

In [5]:
ev = sd.get_event_info(
    origin_time=config['origin_time'],
    base_catalog="USGS",
    magnitude=config['magnitude']
)

Error getting event information from USGS:
Specification mandates value for attribute crossorigin, line 12, column 39 (<string>, line 12)
No event found in USGS catalog for given time. Using dummy event info.


This will load the data from the specified data source:
- local mseed file
- local sds system
- fdsn (online)


In [6]:
sd.load_data(config['tbeg'], config['tend'])

-> fetching PY.PFOIX..HHZ data from FDSN
 -> loading translational data failed!
'NoneType' object has no attribute 'get_waveforms'
-> fetching PY.PFOIX..HHN data from FDSN
 -> loading translational data failed!
'NoneType' object has no attribute 'get_waveforms'
-> fetching PY.PFOIX..HHE data from FDSN
 -> loading translational data failed!
'NoneType' object has no attribute 'get_waveforms'
0 Trace(s) in Stream:

-> fetching PY.BSPF..HJZ data from FDSN
 -> loading rotational data failed!
'NoneType' object has no attribute 'get_waveforms'
-> fetching PY.BSPF..HJN data from FDSN
 -> loading rotational data failed!
'NoneType' object has no attribute 'get_waveforms'
-> fetching PY.BSPF..HJE data from FDSN
 -> loading rotational data failed!
'NoneType' object has no attribute 'get_waveforms'
-> rotating rotational data  to ZNE
0 Trace(s) in Stream:

 -> missing stream data


Ensure that all traces of stream have the same sample size

In [ ]:
sd.trim_stream(set_common=True, set_interpolate=False)

Now we can have a look at the waveforms of all 6 components

In [ ]:
sd.st.plot(equal_scale=False);

There is also an internal waveform plot function

In [ ]:
fig = sd.plot_waveforms() 

### Only required for time lag correction

This computes the time lag between the translation and rotation components based on cross-correlation for transverse and vertical component combinations. If correction is True, the time lag is applied to the rotation component.

In [ ]:
# #finde (and correct) time lag between translation and rotation data
# sd.get_component_lag(normalize=True,
#                      baz=ev['backazimuth'],
#                      correct=True)

# # update endtime
# sd.tend = sd.tbeg+25

# # trim stream
# sd.st = sd.st.trim(sd.tbeg, sd.tend)

# sd.st.plot(equal_scale=False);

### Write data to file or SDS system

This allows to write the data to a file or the SDS system. For the mseed file the seed id is as specified in the config.

In [ ]:
# write to mseed file
# sd.st.write("./data/eventM35_6c.mseed", format="MSEED")

# write to sds filesystem
# sd.write_to_sds(sd.st, "./data/", format="MSEED")

## Analysis

### Wavelet Comparsion

This allows to compute the CWT for all components and plot the results.

In [ ]:
# First compute CWT for all components
cwt_results = {}
for comp in ['Z', 'N', 'E']:
    # Rotation components
    rot_data = sd.get_stream("rotation").select(channel=f"*{comp}")[0]
    cwt_results[f"{rot_data.stats.channel}"] = sixdegrees.compute_cwt(
        rot_data.times(), 
        rot_data.data, 
        rot_data.stats.delta,
        normalize=True
    )
    
    # Translation components
    acc_data = sd.get_stream("translation").select(channel=f"*{comp}")[0]
    cwt_results[f"{acc_data.stats.channel}"] = sixdegrees.compute_cwt(
        acc_data.times(), 
        acc_data.data, 
        acc_data.stats.delta,
        normalize=True
    )


In [ ]:

# Then plot all components
fig = plot_cwt_all(
    rot=sd.get_stream("rotation"),
    acc=sd.get_stream("translation"),
    cwt_output=cwt_results,
    clog=True,
    ylim=50
)

# fig.savefig("cwt_all.png")

Frequency band to filter (this is applied to the stream object after being reset to raw stream)

In [ ]:
sd.filter_data(fmin=0.5, fmax=10.0, output=False)

Apply polarity correction if needed. This is applied to the stream object that is processed. If raw is True, the polarity correction is applied to the raw stream too.

In [ ]:
sd.polarity_stream(pol_dict={"HZ": -1, "JZ": 1}, raw=True)

Let's compare the waveforms of the rotation (red) and translational (black) components. According to the theory both should be in phase for plane waves, thus the cross-correlation should be close to 1.

In [ ]:
fig = sd.plot_waveform_cc(
    twin_sec=1,
    twin_overlap=0.5,
    wave_type="both",
    unitscale="micro",
    t1=UTCDateTime(config['tbeg'])+1,
    t2=UTCDateTime(config['tend'])-15,
)

Let's trim the data to the main event

In [ ]:
sd.trim(UTCDateTime(config['tbeg'])+1, UTCDateTime(config['tend'])-15)

### Plot Spectra of Components

Here we can compare the FFT spetra of the rotation and translational components.

In [ ]:
fig = sd.plot_spectra_comparison_fill(
    rot=sd.get_stream("rotation"), 
    acc=sd.get_stream("translation"),
    fmin=1, 
    fmax=15,
    ylog=False, 
    xlog=False, 
    fill=True
)

### Backazimuth Estimate

Now we want to estimate the backazimuth of the event. We can do this by computing the cross-correlation between the rotation and translational components (representing Love and Rayleigh waves, respectively) for different backazimuths.

In [ ]:
# First compute backazimuth
baz_results = sd.compute_backazimuth(
    wave_type='love',
    baz_step=5,
    baz_win_sec=0.5,
    baz_win_overlap=0.5,
    out=True
)

# Then plot results
fig = sd.plot_backazimuth_results(
    baz_results,
    wave_type='love',
    baz_theo=ev['backazimuth'],
    cc_threshold=0.2,  # optional
)

In [ ]:
# First compute backazimuth
baz_results = sd.compute_backazimuth(
    wave_type='rayleigh',
    baz_step=1,
    baz_win_sec=0.5,
    baz_win_overlap=0.5,
    out=True
)

# # Then plot results
fig = sd.plot_backazimuth_results(
    baz_results,
    wave_type='rayleigh',
    baz_theo=ev['backazimuth'],
    cc_threshold=0.2,  # optional
)

We can also compare the different backazimuth estimation methods. This also includes a method based on the horizontal rotation components (called tangent method). This is then compared to the theoretical backazimuth (based on the event location).

In [ ]:
fig, _results = sd.compare_backazimuth_methods(
    Twin=0.5,
    Toverlap=0.5,
    cc_threshold=0.5,
    baz_theo=ev['backazimuth'],
    plot=True,
    output=True
)

### Analyse phase velocities

6 DoF observations allow us to estimate the phase velocities of Love and Rayleigh waves based on the amplitude ratios. For the estimation we use linear regression methods (ODR and RANSAC) to estimate the slope of rotational and translational amplitudes in time windows.


In [ ]:
# Compute Love wave velocities
love_velocities = sd.compute_velocities(
    wave_type='love',
    win_time_s=1.5,
    overlap=0.5,
    cc_threshold=0.4,
    method='odr',
    baz=ev['backazimuth']  # optional
)

# Compute Rayleigh wave velocities
rayleigh_velocities = sd.compute_velocities(
    wave_type='rayleigh',
    win_time_s=0.5,
    overlap=0.5,
    cc_threshold=0.4,
    method='odr',
    baz=ev['backazimuth']  # optional
)

We can plot the results with the waveforms which are rotated using the theoretical backazimuth.

In [ ]:
# Plot results
fig = sd.plot_velocities(
    velocity_results=love_velocities,
    vmax=5000,  # optional
    minors=True # optional
)

# Plot results
fig = sd.plot_velocities(
    velocity_results=rayleigh_velocities,
    vmax=5000,  # optional
    minors=True # optional
)

### Optimization of parameters

This method allows to optimize the parameters for the velocity estimation. We scan frequency bands (the time window is then computed automatically) and backazimuths in a grid search and then select the best parameters based on the cross-correlation that are used for the velocity estimates.

In [ ]:
# Get optimization results
params = sd.optimize_parameters(
    wave_type='love', # select wave type
    fbands={'fmin': 1, 'fmax': 10.0, 'octave_fraction': 6}, # frequency band specification
    baz_step=1,  # step size for the backazimuth search
    twin_min=1,  # minimum time window in seconds
    overlap=0.9,  # overlap of the time windows
    bandwidth_factor=2  # bandwidth factor for the frequency band
)

# Plot results using the class method
fig = sd.plot_optimization_results(
    params, 
    wave_type='love',
    vel_max_threshold=5000,
    cc_threshold=0.5,
    baz_theo=ev['backazimuth']
)


In [ ]:
# Get optimization results
params = sd.optimize_parameters(
    wave_type='rayleigh', # select wave type
    fbands={'fmin': 0.1, 'fmax': 1.0, 'octave_fraction': 6}, # frequency band specification
    baz_step=1,  # step size for the backazimuth search
    twin_min=1,  # minimum time window in seconds
    overlap=0.5,  # overlap of the time windows
    bandwidth_factor=2  # bandwidth factor for the frequency band
)

# Plot results using the class method
fig = sd.plot_optimization_results(
    params, 
    wave_type='rayleigh',
    vel_max_threshold=5000,
    cc_threshold=0.8,
    baz_theo=ev['backazimuth']
)
